<a href="https://colab.research.google.com/github/leodenale/SEOData4RecipeKeywords-GandYT/blob/master/SEOData4RecipeKeyword.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SEO Data for Recipe Keywords (Google & YouTube)
## 484 queries' results on Google and YouTube, with metadata

I've written previously on search data and what can be done with it for different industries:

- SEMrush article [analyzing tickets and flights keywords on Google search](https://www.semrush.com/blog/analyzing-search-engine-results-pages/)
- [Cars for sale on Google search](https://www.kaggle.com/eliasdabbas/search-engine-results-pages-serps-research) (explains more on how to use the [`serp_goog` function](https://advertools.readthedocs.io/en/master/advertools.html#module-advertools.serp))
- [US 2018 Midterm Elections candidates on Google Search](https://www.kaggle.com/eliasdabbas/us-midterm-elections-2018-on-google-search)
- You can also generate your own similar dataset through this [SEO keyword SERP dashboard](https://www.dashboardom.com/google-serp) if you want

Here, I go through the same, but for a different industry (recipes and food), and I also include data from YouTube search results.  
YouTube provides much richer data about the search results, mainly because it hosts the content and the engagement on the platform. On regular search, you basically have one metric, which is the rank of the keyword, but on YouTube, you have a lot of data on views, comments, likes, etc. for the video, as well as a lot of metadata about the channel that produced the video; the number of videos they have, subscribers, total views, and so on. 

I'll be using `pandas` for data manipulation, `plotly` for data visualization, and `advertools`, for importing search data (and having it in a DataFrame, analyzing the text (titles, descriptions, etc.), and how it relates to numbers (views, likes, subscribers, etc.).

In [2]:
pip install advertools

     |████████████████████████████████| 256kB 5.5MB/s 
     |████████████████████████████████| 245kB 7.3MB/s 
     |████████████████████████████████| 2.6MB 8.5MB/s 
     |████████████████████████████████| 3.2MB 25.1MB/s 
     |████████████████████████████████| 245kB 32.6MB/s 
     |████████████████████████████████| 3.1MB 37.3MB/s 
     |████████████████████████████████| 61kB 6.2MB/s 
     |████████████████████████████████| 81kB 6.6MB/s 
     |████████████████████████████████| 61kB 7.1MB/s 
  Created wheel for protego: filename=Protego-0.1.16-cp36-none-any.whl size=7766 sha256=6208da19621ff39bb1f9bf60532686599c1a38b6279ecf56b4faeb74bfe513a0
  Stored in directory: /root/.cache/pip/wheels/51/01/d1/4a2286a976dccd025ba679acacfe37320540df0f2283ecab12
  Created wheel for PyDispatcher: filename=PyDispatcher-2.0.5-cp36-none-any.whl size=11516 sha256=04794c882e3c97a91d543b5d2c307c910cabab851dc407d9f39c485c62012d2f
  Stored in directory: /root/.cache/pip/wheels/88/99/96/cfef6665f9cb1522ee6757ae59

In [3]:
import pandas as pd
pd.options.display.max_columns = None
import plotly
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot, plot
init_notebook_mode(connected=True)

import advertools as adv
print('Package        Version')
print('='*25)
for p in [adv, pd, plotly]:
    print(f"{p.__name__:<15}", ': ', p.__version__, sep='')

Package        Version
advertools     : 0.10.7
pandas         : 1.1.5
plotly         : 4.4.1


This is the code to get the national dishes from [Wikipedia's page](https://en.wikipedia.org/wiki/National_dish). This list has a valid criticism (which is also mentioned on the page), about how representative it is. There are countries with populations of over a billion people, who have one or two recipes each, and countries with a population less than ten million who also have their national dish listed. Keep this in mind while thinking about the results. 

In [7]:
import requests
from bs4 import BeautifulSoup
page = 'https://en.wikipedia.org/wiki/National_dish'
resp = requests.get(page)
soup = BeautifulSoup(resp.text, 'lxml')
dishlist = [x.text.strip() for x in soup.select('a + b')]
dishes_df = pd.DataFrame(dishlist, columns=['dish'])
dishes_df.to_csv('recipes_dish_list.csv', index=False)

In [8]:
dishes = pd.read_csv('../input/recipes-search-engine-results-data/recipes_dish_list.csv')
dishes.head()

FileNotFoundError: ignored

To generate the queries, I added "recipe" and "how to make" to each dish.  
As a result each dish will be searched twice, once as "**dish** recipe" and once as "how to make 
**dish**".
    

In [ ]:
queries_recipe = [x + ' recipe' for x in dishes['dish'].tolist()]
queries_how_to_make = ['how to make ' + x for x in dishes['dish'].tolist()]
queries = queries_recipe + queries_how_to_make
queries[:10] + queries[-10:]

To get the data, all you have to run is one line of code for Google and for YouTube. 
The functions `serp_goog` and `serp_youtube` take a lot of optional arguments to customize your queries. Here we simply give it a list of the queries that we created above. The funtion loops through them all and handles concatenating them into one DataFrame, ready for analysis. If you add other parameters, the function will request data for the **product** of all parameters that you provide.  
For example, if you have as queries ['pizza recipe', 'how to make pizza'], then two queries will be generated.  
If in addition to `q` you provide a list of countries, let's say ['us', 'ca', 'uk'], then each query will be run three times, once for each country. As a result in this case you will have six requests. You can imagine how this can grow exponentially if you provide a lot of parameters. 

### Google API Setup

1. [Create a custom search engine](https://cse.google.com/cse/all). At first, you might be asked to enter a site to search. Enter any domain, then go to the control panel and remove it. Make sure you enable "Search the entire web" and image search. You will also need to get your search engine ID, which you can find on the control panel page.
2. [Enable the custom search API](https://console.cloud.google.com/apis/library/customsearch.googleapis.com). The service will allow you to retrieve and display search results from your custom search engine programmatically. You will need to create a project for this first.
3. [Create credentials for this project so you can get your key](https://console.developers.google.com/apis/api/customsearch.googleapis.com/credentials).
4. [Enable billing for your project](https://console.cloud.google.com/billing/projects) if you want to run more than 100 queries per day. The first 100 queries are free; then for each additional 1,000 queries, you pay USD $5.


In [ ]:
# Code to get the data:
cx = 'YOUR_GOOGLE_CUSTOM_SEARCH_ENGINE'
key = 'YOUR_GOOGLE_DEVELOPER_API_KEY'

# recipes = adv.serp_goog(cx=cx, key=key, q=queries)
# recipes_ytb = adv.serp_youtube(key=key, q=queries)

Here is a sample of the Google data showing the first eight columns, which are the most important. Feel free to explore other columns if you want.

In [ ]:
recipes = pd.read_csv('../input/recipes-search-engine-results-data/recipes_serp_data.csv')
print('shape:', recipes.shape)
recipes.sample(5).iloc[:, :8]

I think column names are clear. Just keep in mind that `queryTime` refers to when this query was requested from Google. This becomes very important if you plan on running the same queries periodically, to show how rankings are changing in time. 

Now let's create a quick summary of all the domains that appeared, and some summary statistics: 

- `count`: The number of times the domain appeared in the dataset. Keep in mind that the same domain might appear more than once, in the same query. As you can see below YouTube appeared 154% of the queries (662 ÷ 430 unique queries).
- `avg_rank`: The average rank that the domain appeared on, in the dataset. 
- `coverage`: The number of times the domain appeared ÷ the number of unique queries sent.

In [ ]:
summary = (recipes
           .groupby(['displayLink'], as_index=False)
           .agg({'rank': ['count', 'mean']})
           .sort_values(('rank', 'count'), ascending=False)
           .assign(coverage=lambda df: df[('rank', 'count')].div(recipes['searchTerms'].nunique())))
summary.columns = ['displayLink', 'count', 'avg_rank', 'coverage']
summary['displayLink'] = summary['displayLink'].str.replace('www.', '')
summary['avg_rank'] = summary['avg_rank'].round(1)
summary['coverage'] = summary['coverage'].mul(100).round(1).astype(str).add('%')
summary.head(20).reset_index(drop=True)

YouTube is clearly the undisputed leader, and very far from the first competitor. 
Let's visualize this summary for a different view. Before that, we create a summary showing how many times each domain appeared on every rank.  

For example, in the sample below, cooking.nytimes.com appeared on rank one, eight times, on rank two, five times, and so on. Before that I created `top_domains` which are the top ten domains that appeared the most. You can change this of course if you want more, or fewer domains.

In [ ]:
top_domains = recipes['displayLink'].value_counts()[:10].index.tolist()
top_df = recipes[recipes['displayLink'].isin(top_domains)]

rank_counts = top_df.groupby(['displayLink', 'rank']).agg({'rank': ['count']}).reset_index()
rank_counts.columns = ['displayLink', 'rank', 'count']
rank_counts.head()

In [ ]:
fig = go.FigureWidget()

fig.add_scatter(x=top_df['displayLink'].str.replace('www.', ''),
                y=top_df['rank'], mode='markers',
                marker={'size': 35, 'opacity': 0.035,})

fig.add_scatter(x=rank_counts['displayLink'].str.replace('www.', ''),
                y=rank_counts['rank'], mode='text', text=rank_counts['count'])

fig.layout.hovermode = False
fig.layout.yaxis.autorange = 'reversed'
fig.layout.yaxis.zeroline = False
fig.layout.yaxis.tickvals = list(range(1, 11))
fig.layout.height = 600
fig.layout.title = 'Top Domains for 430 Recipes Keywords - Google'
fig.layout.yaxis.title = 'SERP Rank (number of appearances)'
fig.layout.showlegend = False
fig.layout.paper_bgcolor = '#eeeeee'
fig.layout.plot_bgcolor = '#eeeeee'
iplot(fig)

For every appearance on a search result, an almost transparent circle is plotted on that position for the domain. The more times it appears, the more opaque the circle gets. As you can expect, YouTube's results clearly stand out from the rest.  
The numbers on the circles show the exact number of times that each domain appeared on that position (rank). YouTube appears mostly on ranks three to five, as shown above, in the summary table where they have an average rank of 4.8.

### Comparison with flights keywords 

To get an idea of how this distribution compares to other industries, here is a quick look at a similar data set for flights and tickets keywords. This is for one hundred top destinations (cities), and two keyword variations for each.  
The below code is pretty much the same as the one we ran for recipes.

In [ ]:
flights_serp = pd.read_csv('../input/search-engine-results-flights-tickets-keywords/flights_tickets_serp2018-12-16.csv')

flights_serp = flights_serp[flights_serp['gl'] == 'us']
flights_serp.iloc[:, :8].sample(5)

In [ ]:
summary_flights = (flights_serp
                   .groupby(['displayLink'], as_index=False)
                   .agg({'rank': ['count', 'mean']})
                   .sort_values(('rank', 'count'), ascending=False)
                   .assign(coverage=lambda df: df[('rank', 'count')].div(flights_serp['searchTerms'].nunique())))

summary_flights.columns = ['displayLink', 'count', 'avg_rank', 'coverage']
summary_flights['displayLink'] = summary_flights['displayLink'].str.replace('www.', '')
summary_flights['avg_rank'] = summary_flights['avg_rank'].round(1)
summary_flights['coverage'] = summary_flights['coverage'].mul(100).round(1).astype(str).add('%')
summary_flights.head(10).reset_index(drop=True)

This looks quite different from how recipes domains are distributed on search. YouTbe has coverage of 154%, and the first competitor is at 41%. Here, there is fierce competition among the top four, all of which are almost 80% and above. 

In [ ]:
top_domains_flights = flights_serp['displayLink'].value_counts()[:10].index.tolist()
top_df_flights = flights_serp[flights_serp['displayLink'].isin(top_domains_flights)]

rank_counts_flights = top_df_flights.groupby(['displayLink', 'rank']).agg({'rank': ['count']}).reset_index()
rank_counts_flights.columns = ['displayLink', 'rank', 'count']
rank_counts_flights.head()

In [ ]:
fig = go.FigureWidget()

fig.add_scatter(x=top_df_flights['displayLink'].str.replace('www.', ''),
                y=top_df_flights['rank'], mode='markers',
                marker={'size': 35, 'opacity': 0.035,})


fig.add_scatter(x=rank_counts_flights['displayLink'].str.replace('www.', ''),
                y=rank_counts_flights['rank'], mode='text', text=rank_counts_flights['count'])

fig.layout.hovermode = False
fig.layout.yaxis.autorange = 'reversed'
fig.layout.yaxis.zeroline = False
fig.layout.yaxis.tickvals = list(range(1, 11))
fig.layout.height = 600
fig.layout.title = 'Top Domains for Flights and Tickets Keywords - Google - USA'
fig.layout.yaxis.title = 'SERP Rank (number of appearances)'
fig.layout.showlegend = False
fig.layout.paper_bgcolor = '#eeeeee'
fig.layout.plot_bgcolor = '#eeeeee'
iplot(fig)

## Recipe Keywords on YouTube

In [ ]:
recipes_ytb = pd.read_csv('../input/recipes-search-engine-results-data/recipes_serp_youtube_data.csv')
print('shape:', recipes_ytb.shape)
recipes_ytb.sample(5)

Note the difference in the number of columns here. Ninety compared to twenty seven on Google search.  
There are several numeric columns that we can use to analyze YouTube data. Here they are (I excluded `video.favoriteCount` and `channel.commentCount` because they are all zeros in this case.

In [ ]:
count_columns = [col for col in recipes_ytb.columns if 'Count' in col if col not in ['video.favoriteCount', 'channel.commentCount']]
count_columns

I wanted to quickly check if there is any correlation between any of those numeric columns and the rank of the channel. Apparently there seems to be none!

In [ ]:
for col in count_columns:
    corr_df = recipes_ytb[['rank', col]].corr()
    print(f"{col:>30} {corr_df['rank'][-1]:%}")

In [ ]:
summary_ytb = (recipes_ytb
              .groupby(['channelTitle'], as_index=False)
              .agg({'rank': ['count', 'mean']})
              .sort_values(('rank', 'count'), ascending=False)
              .assign(coverage=lambda df: df[('rank', 'count')].div(recipes_ytb['q'].nunique())))
summary_ytb.columns = ['channelTitle', 'count', 'avg_rank', 'coverage']
# summary_ytb['displayLink'] = summary_ytb['displayLink'].str.replace('www.', '')
summary_ytb['avg_rank'] = summary_ytb['avg_rank'].round(1)
summary_ytb['coverage'] = summary_ytb['coverage'].mul(100).round(1).astype(str).add('%')
summary_ytb.head(10).reset_index(drop=True)

Here the picture is completely different from Google search (for both, recipes and flights). The top channel has a coverage score of 4.6%.  
Remember that by default YouTube returns five results per query, so this also has an effect on how data are distributed. Feel free to experiment with more results if you want. 

In [ ]:
top_channels = recipes_ytb['channelTitle'].value_counts()[:10].index.tolist()
top_channels_df = recipes_ytb[recipes_ytb['channelTitle'].isin(top_channels)]

rank_counts_ytb = top_channels_df.groupby(['channelTitle', 'rank']).agg({'rank': ['count']}).reset_index()
rank_counts_ytb.columns = ['channelTitle', 'rank', 'count']
rank_counts_ytb.head()

In [ ]:
fig = go.FigureWidget()

fig.add_scatter(x=top_channels_df['channelTitle'],
                y=top_channels_df['rank'], mode='markers',
                marker={'size': 35, 'opacity': 0.25,})


fig.add_scatter(x=rank_counts_ytb['channelTitle'].str.replace('www.', ''),
                y=rank_counts_ytb['rank'], mode='text', text=rank_counts_ytb['count'])

fig.layout.hovermode = False
fig.layout.yaxis.autorange = 'reversed'
fig.layout.yaxis.zeroline = False
fig.layout.yaxis.tickvals = list(range(1, 11))
fig.layout.height = 600
fig.layout.title = 'Recipes Keywords Rankings - YouTube'
fig.layout.yaxis.title = 'SERP Rank (number of appearances)'
fig.layout.showlegend = False
fig.layout.paper_bgcolor = '#eeeeee'
fig.layout.plot_bgcolor = '#eeeeee'
iplot(fig)

## Text Analysis

Since we have a lot of numeric columns that describe videos and channels, and since we also have many text columns, let's do some word counting and see which are the most common.  
Note that there are two types of counting that are provided by the `word_frequency` function. 

- Absolute (`abs_freq`): This is a simple count of how many times each word was used in the text list that we specify. 
- Weighted (`wtd_freq`): This takes into consideration the numeric value that we provide. For example, if a video has the title "sushi recipe" and that video got one thousand views, then "sushi" and "recipe" would be counted a thousand times each. This shows how important the word was from a consumption perspective and not just from a production point of view. I have another article on [text analysis for online marketers](https://www.semrush.com/blog/text-analysis-for-online-marketers/) if you are interested.

In [ ]:
serp_word_freq = adv.word_frequency(recipes['title'], 
                                    rm_words=list(adv.stopwords['english']) + ['-', '', '|', '–', '&'])
serp_word_freq.head(10)

The above table shows the most used words in the title column of the `recipes` DataFrame, for Google. There is no numeric value (other than the rank) so, without providing a `num_list` argument, the frequency defaults to one.  
"recipe", "food", "youtube", and "recipes" (the top words) are not surprising. But for me, having "chicken" as the top specific food word is interesting. It is almost twice as much as beef. I wouldn't expect rice to be that prominent either. What do you expect? 

Below is the same count for the snippet column. Again chicken, together with rice, are the only specific foods mentioned. The rest are generic words about recipes and food. 

In [ ]:
serp_word_freq_snippet = adv.word_frequency(recipes['snippet'].fillna(''), rm_words=list(adv.stopwords['english']) + ['-', '', '|', '–', '&'])
serp_word_freq_snippet.head(10)

Things get more interesting with weighted frequency.  
Here is the word frequency for YouTube's DataFrame, counting words in the title column, with videos' number of views.

In [ ]:
serp_ytb_word_freq_title = adv.word_frequency(text_list=recipes_ytb['title'].fillna(''), 
                                              num_list=recipes_ytb['video.viewCount'],
                                              rm_words=list(adv.stopwords['english']) + ['-', '', '|', '–', '&'])
serp_ytb_word_freq_title.head(10).style.format({'wtd_freq': '{:,.0f}', 'rel_value': '{:,.0f}'})

The DataFrame is sorted by `wtd_freq` by default but you can obviously change that if you want.  
"recipe" was used 412 times, and the sum total of views for all videos who's title contains "recipes" was 394,834,312.  
The words on indexes two to six are interesting. They all have an `abs_freq` of one, and the same number on `wtd_freq`. This means that there was one video who's title contained these words, and this is further confirmed by the fact that they all have the same `wtd_freq`.  
This is a very important example, where one row can affect the whole dataset. This is typical on social media where you have people with tens of millions of followers, and with one post, can make such a dramatic difference to the averages.  
Again "chicken" is the top specific food word in this DataFrame. 

Let's run the same function for the description column (also weighted by the number of views):

In [ ]:
serp_ytb_word_freq_desc = adv.word_frequency(recipes_ytb['description'].fillna(''), recipes_ytb['video.viewCount'],
                                              rm_words=list(adv.stopwords['english']) + ['-', '', '|', '–', '&'])
serp_ytb_word_freq_desc.head(20).style.format({'wtd_freq': '{:,.0f}', 'rel_value': '{:,.0f}'})

#### Video tags

In [ ]:
serp_ytb_word_freq_tags = adv.word_frequency(recipes_ytb['video.tags'].fillna(''), recipes_ytb['video.viewCount'],
                                              rm_words=list(adv.stopwords['english']) + ['-', '', '|', '–', '&'])
serp_ytb_word_freq_tags.head(20).style.format({'wtd_freq': '{:,.0f}', 'rel_value': '{:,.0f}'})

#### Video Topics

In [ ]:
serp_ytb_word_freq_topics = adv.word_frequency(recipes_ytb['video.topicCategories'].fillna(''), recipes_ytb['video.viewCount'],
                                              rm_words=list(adv.stopwords['english']) + ['-', '', '|', '–', '&'])
serp_ytb_word_freq_topics.head(20).style.format({'wtd_freq': '{:,.0f}', 'rel_value': '{:,.0f}'})

Another option that is available through the `word_frequency` function is that you can provide a regular expression to extract specific patterns of words.  
Let's say you want to know which/if hashtags are popular in video descriptions. All you have to do is supply the regex for that and the function will extract them, and only run the counts for those extracted words.  
The `regex` module in `advertools` provides a set of regexes that can be used.  
Here we use `adv.regex.HASHTAG_RAW` to extract the hashtags. 

In [ ]:
(adv.word_frequency(recipes_ytb['video.description'].fillna(''), 
                    recipes_ytb['video.viewCount'],
                    regex=adv.regex.HASHTAG_RAW,
                    rm_words=list(adv.stopwords['english']) + ['-', '', '|', '–', '&'])
 .head(15)
 .style.format({'wtd_freq': '{:,.0f}', 'rel_value': '{:,.0f}'}))

### @mentions

In [ ]:
(adv.word_frequency(recipes_ytb['video.description'].fillna(''), 
                    recipes_ytb['video.viewCount'],
                    regex=adv.regex.MENTION_RAW,
                    rm_words=list(adv.stopwords['english']) + ['-', '', '|', '–', '&'])
 .head(15)
 .style.format({'wtd_freq': '{:,.0f}', 'rel_value': '{:,.0f}'}))

### Emoji

In [ ]:
(adv.word_frequency(recipes_ytb['video.description'].fillna(''), 
                   recipes_ytb['video.viewCount'],
                   regex=adv.emoji_dict.emoji_regexp,
                   rm_words=list(adv.stopwords['english']) + ['-', '', '|', '–', '&'])
 .head(15)
 .assign(emoji_text=lambda df: [adv.emoji_dict.emoji_dict[x] for x in df['word']])
 .style.format({'wtd_freq': '{:,.0f}', 'rel_value': '{:,.0f}'}))

### Currency Symbols

In [ ]:
(adv.word_frequency(recipes_ytb['video.description'].fillna(''), 
                    recipes_ytb['video.viewCount'],
                    regex=adv.regex.CURRENCY_RAW,
                    rm_words=list(adv.stopwords['english']) + ['-', '', '|', '–', '&'])
 .head(15)
 .style.format({'wtd_freq': '{:,.0f}', 'rel_value': '{:,.0f}'}))

This was a quick overview of doing some SEO research on Google and YouTube, and how you can compare, and extract certain information.  
There is more that can be done but these are some of the main features that I think are important.  
If you have any suggestions/questions, or come across any bugs, [let me know.](https://github.com/eliasdabbas/advertools/issues)